# 分割表の統計解析 Chapter 1
## 目的
- 説明変数と目的変数の関連性（カイ二乗適合度検定,尤度比検定）
- 説明変数と目的変数の関連性の度合（コレスポンデンス分析）
- 処理群と対照群の差
- 交絡因子の解明によるシンプソンパラドックスの抑制

In [224]:
import numpy as np
import scipy as sp
import pandas as pd

## TODO
- sum_allメソッドの結果はキャッシュにする
- 自己完結可能な値はInitializerで定義する
    - メソッドに引数がないものはInitializerで値を算出してしまえばよい

In [236]:
class ContingencyTableAnalysis:
    """
    分割表の統計解析モジュール。
    
    Parameters
    ----------
    df_cont_tb: pd.DataFrame
        解析元の分割表。
    
    sum_raw_name: str, default 'raw_all'
        分割表の行和の項目名。
    
    sum_col_name: str, default 'col_all'
        分割表の列和の項目名。
    
    const_law: 'all' or 'raw_all' or 'col_all' or 'none'
        分割表中の確定した値の箇所。
        'all': 行列和が確定的
        'raw_all': 行和が確定的
        'col_all': 列和が確定的
        'none': 全セルが確定的
    """
    
    # Const
    
    _CONST_LAWS = ['all', 'raw_all', 'col_all', 'none']
    _ERROR_CONST_LAW = "const_law must be inputted 'all' or 'raw_all' or 'col_all', or 'none'."
    
    def __init__(self, df_cont_tb: pd.DataFrame, sum_raw_name='raw_all', sum_col_name='col_all', const_law='all'):
        if const_law not in self._CONST_LAWS:
            raise AttributeError(self._ERROR_CONST_LAW)
            
        self.df_cont_tb = df_cont_tb
        self.sum_raw_name = sum_raw_name
        self.sum_col_name = sum_col_name
        self.const_law = const_law
    
    # Public
    
    def sum_all(self) -> pd.DataFrame:
        """
        行和、列和、行列和を計算したDataFrameを返す。
        
        Returns
        -------
        df_add_sum: pd.DataFrame
            初期化引数df_cont_tbに対して行和、列和、行列和を計算したDataFrame。
        """
        df_add_sum = self.df_cont_tb.copy()
        df_add_sum[self.sum_col_name] = df_add_sum.sum(axis='columns')
        df_add_sum = self._sum_row(df_add_sum, self.sum_raw_name)
        return df_add_sum
    
    def maximum_likelihood_estimate(self) -> pd.DataFrame:
        """
        分割表の最尤推定値を計算したDataFrameを返す。
        
        Returns
        -------
        df_estimated: pd.DataFrame
            初期化引数df_cont_tbの最尤推定値を計算したDataFrame。
        """
        data_raw_sum = self.df_cont_tb.sum(axis='index')
        data_col_sum = self.df_cont_tb.sum(axis='columns')
        data_all_sum = data_raw_sum.sum()
        
        estimated_data = []
        for r_sum in data_col_sum:
            estimated_col = []
            for c_sum in data_raw_sum:
                estimated_col.append(r_sum * c_sum / data_all_sum**2)
            estimated_data.append(estimated_col)
        
        df_estimated = pd.DataFrame(estimated_data, self.df_cont_tb.index, self.df_cont_tb.columns)
        return df_estimated

    
    def get_expected_frequency(self) -> pd.DataFrame:
        """
        分割表の推定度数を計算したDataFrameを返す。
        
        Returns
        -------
        df_freq: pd.DataFrame
            初期化引数df_cont_tbの推定度数を計算したDataFrame。
        """
        data_raw_sum = self.df_cont_tb.sum(axis='index')
        data_col_sum = self.df_cont_tb.sum(axis='columns')
        data_all_sum = data_raw_sum.sum()
        
        freq_data = []
        for r_sum in data_col_sum:
            freq_col = []
            for c_sum in data_raw_sum:
                freq_col.append(r_sum * c_sum / data_all_sum)
            freq_data.append(freq_col)
        
        df_freq = pd.DataFrame(freq_data, self.df_cont_tb.index, self.df_cont_tb.columns)
        return df_freq
    
    def chi2_fit_test(self, per_point: float) -> dict:
        """
        カイ二乗適合度検定を実行。
        分割表の行項目と列項目に対して独立性の帰無仮説を検証する。
        
        Parameters
        ----------
        per_point: float object
            カイ二乗分布の上側確率（実数）。
        
        Returns
        -------
        test_info: dict included in 'chi2_fit', 'chi2_per', 'test_result'
            'chi2_fit'は推定度数によるカイ二乗適合度の値。
            'chi2_per'は指定したカイ二乗分布の上側確率におけるパーセント点。
            'test_result'は帰無仮説が棄却される場合False、棄却されない場合True。
        """
        df_src = self.df_cont_tb.copy()
        df_est = self.get_expected_frequency()
        
        # chi2 fitting
        df_fit = (df_src - df_est)**2 / df_est
        chi2_fit = df_fit.sum(axis='columns').sum()
        
        # chi2 percentage point
        df = (len(self.df_cont_tb.index) - 1) * (len(self.df_cont_tb.columns) - 1)
        chi2_per = sp.stats.chi2.ppf(q=per_point, df=df)
                          
        # testing
        test_result = False if chi2_fit > chi2_per else True
        
        test_info = {
            'chi2_fit': chi2_fit,
            'chi2_per': chi2_per,
            'test_result': test_result
        }
        return test_info
    
    def likelihood_ratio_test():
        pass

    def get_all(self):
        """
        モジュール内結果を全て取得する。
        """
        pass
        
    # Private
    
    def _sum_row(self, df: pd.DataFrame, raw_name: str) -> pd.DataFrame:
        df_result = df.copy()
        df_row_sum = pd.DataFrame(df.sum(axis='index')).T
        df_row_sum.index = [raw_name]
        return df_result.append(df_row_sum)

    def _get_all_sum(self, df_sum_all: pd.DataFrame) -> int:
        return df_sum_all.loc[self.sum_raw_name, self.sum_col_name]

## 行列和が確定的
- 代数の合計と微積の合計の総合計145のみが確定的

In [241]:
grades = ['優', '良', '可', '不可']
df_math_grade = pd.DataFrame(
    [[18, 7, 1, 2], [16, 13, 17, 12], [7, 3, 10, 21], [7, 2, 3, 6]],
    ['微積' + '_' + grade for grade in grades],
    ['代数' + '_' + grade for grade in grades]
)
cta_math_grade = ContingencyTableAnalysis(df_math_grade, '微積_合計', '代数_合計', 'all')
df_sum = cta_math_grade.sum_all()
df_estimated = cta_math_grade.maximum_likelihood_estimate()
df_freq = cta_math_grade.get_expected_frequency()
df_test_result = cta_math_grade.chi2_fit_test(0.99)
display(df_math_grade)
display(df_sum)
display(df_estimated)
display(df_freq)
display(df_test_result)

,代数_優,代数_良,代数_可,代数_不可
微積_優,18,7,1,2
微積_良,16,13,17,12
微積_可,7,3,10,21
微積_不可,7,2,3,6


,代数_優,代数_良,代数_可,代数_不可,代数_合計
微積_優,18,7,1,2,28
微積_良,16,13,17,12,58
微積_可,7,3,10,21,41
微積_不可,7,2,3,6,18
微積_合計,48,25,31,41,145


,代数_優,代数_良,代数_可,代数_不可
微積_優,0.063924,0.033294,0.041284,0.054602
微積_良,0.132414,0.068966,0.085517,0.113103
微積_可,0.093603,0.048751,0.060452,0.079952
微積_不可,0.041094,0.021403,0.026540,0.035101


,代数_優,代数_良,代数_可,代数_不可
微積_優,9.268966,4.827586,5.986207,7.917241
微積_良,19.200000,10.000000,12.400000,16.400000
微積_可,13.572414,7.068966,8.765517,11.593103
微積_不可,5.958621,3.103448,3.848276,5.089655


{'chi2_fit': 36.35368807073809,
 'chi2_per': 21.665994333461924,
 'test_result': False}

## 行和または列和が確定的